In [1]:
import os
import getpass
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass()
client = OpenAI()

#provide the question to parse in 
input = "list me relevant experience"

In [2]:
from pinecone import Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key=getpass.getpass())

c:\Users\shuji\anaconda3.1\envs\proj1\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#!pip install langchain_pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders.csv_loader import CSVLoader

#Recursive character split : divides the input text into smaller chunks of similar sizes in a hierarchical and iterative manner using a set of separators. 
loader = PyPDFLoader("Final resume_Liang Shu-Jing.pdf")
documents= loader.load()
string_ = ''
for i in range(len(documents)):
    string_+=documents[i].page_content
string_
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_text(string_)

In [7]:
import time

index_name = "langchain-test-index"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

Adding items to vector store

In [8]:
from uuid import uuid4
from langchain_core.documents import Document

documents = []

for i in range(len(docs)):

    documents.append(Document(
        page_content=docs[i],
        metadata={"source": "resume"},
    ))

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['a345e264-c93b-45e1-aea5-23145b308324',
 'e83fec39-d88c-493c-8fe8-5fe3ad8d8ea6',
 '318c5257-9465-40be-b507-530ba79164d4',
 '9f8d099d-a8b1-490a-8f39-47ebad80deae',
 '4c83340b-4baa-4820-aae6-a07337489346',
 'a1af1e38-a09b-412c-bb90-486b8011a962',
 '50b5bcfa-d2b4-40cc-930e-cd5ec911cc6c',
 '25c7a72f-f72f-485a-941d-4c7e7f84288d',
 '2a1b5cc2-4ec2-4139-998d-f52b18e69aa0',
 '54308ccc-1789-4785-8834-23de1d141cd2',
 '26bb1078-f6f5-4fe0-978c-5b1889b04717',
 '3f8d0b4a-0190-4e48-8c6e-267b5624fc2c',
 '06b12442-030b-4337-abd8-596e9fa4ebe0',
 '34c41138-cd13-442d-a5ec-ad9d3639fc59',
 'd4995e46-18b2-4011-aa31-72ffd1548ec9',
 '4dfec3c3-6bf2-4e3a-9d86-d3c7ccc6c0a0',
 '61ee78df-eed4-4df4-af37-69de6e47cad9',
 '505aa78c-5b47-4661-a648-ae3c012e2e47',
 '3defd95b-903a-497d-8a68-e2889cc7eb1c',
 'dd393cee-0a01-4b66-a9ec-c56704066026',
 'b3d9b0ed-db46-45e0-9407-a4a0eb41332d',
 '5bbe49ca-16a0-4b56-8c3b-19108560383e',
 '18c6cde6-3621-4428-8807-09355ec75e19',
 'a0ccd506-3c18-4808-9a71-0c7ba331bae2',
 '0eb7b90a-7bf0-

In [31]:
documents

[Document(metadata={'source': 'resume', 'text': 'LIANG SHU-JING\nshujing1999431@gmail.com +65 98557101\nEDUCATION'}, page_content='LIANG SHU-JING\nshujing1999431@gmail.com +65 98557101\nEDUCATION'),
 Document(metadata={'source': 'resume', 'text': 'EDUCATION\nSINGAPORE MANAGEMENT UNIVERSITY Jan 2024 - Present'}, page_content='EDUCATION\nSINGAPORE MANAGEMENT UNIVERSITY Jan 2024 - Present'),
 Document(metadata={'source': 'resume', 'text': 'Masters of IT in Business (Artificial Intelligence)'}, page_content='Masters of IT in Business (Artificial Intelligence)'),
 Document(metadata={'source': 'resume', 'text': 'Relevant modules taken : Data science in Business , Applied Machine learning , R statistics for'}, page_content='Relevant modules taken : Data science in Business , Applied Machine learning , R statistics for'),
 Document(metadata={'source': 'resume', 'text': ', R statistics for Data analysis, Deep'}, page_content=', R statistics for Data analysis, Deep'),
 Document(metadata={'source

Define retrieval

In [13]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.5},
)


include chat history

In [30]:
from typing import Sequence
import streamlit as st
from langchain_core.messages import HumanMessage
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import Annotated, TypedDict

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chat_history = []
def generate_ans(query, llm):

    ### Contextualize question ###
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


    # history_aware_retriever.invoke({"input": "hello", "chat_history": []})

    ### Answer question ###
    system_prompt = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        "{context}"
    )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    for i in rag_chain.stream({"input": query, "chat_history": chat_history}):
        yield i.get("answer")
    # return ai_msg['answer']

for i in generate_ans('What experience doe shujing have', llm):
    print(i)

#     ai_msg= rag_chain.invoke({"input": query, "chat_history": chat_history}).content[0] #check if need this
#     chat_history.extend([HumanMessage(content=question), ai_msg["answer"]])
#     return ai_msg, chat_history


# # Streamlit UI
# st.title("Ask me anything")
# openai_api_key = st.sidebar.text_input("OpenAI API Key", type="password")
# # chat_history = []
# # # User input
# # query_in = st.text_input("Enter your query:")
# # st.session_state.queries.append(query_in)
# # result = generate_ans(query_in)
# # st.session_state.results.append(result)



# with st.form("my_form"):
#     text = st.text_area(
#         "Enter text:",
#         "What are the three key pieces of advice for learning how to code?",
#     )
#     submitted = st.form_submit_button("Submit")
#     if not openai_api_key.startswith("sk-"):
#         st.warning("Please enter your OpenAI API key!", icon="⚠")
#     if submitted and openai_api_key.startswith("sk-"):
#         generate_ans(text)


# "chat_history": [
#     HumanMessage(),
#     AIMessage(),
# ]






None
None

Sh
u
-J
ing
 has
 experience
 working
 at
 AG
ILE
AL
GO
 in
 Singapore
 since
 August
 
202
4
.
 Additionally
,
 they
 are
 pursuing
 a
 Bachelor
 of
 Engineering
 in
 Chemical
 Engineering
.



In [ ]:
import streamlit as st

st.title("Ask me anything")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

if "messages" not in st.session_state:
    st.session_state.messages = []

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("What is up?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        stream = generate_ans(prompt, llm)
        response = st.write_stream(stream)
    st.session_state.messages.append({"role": "assistant", "content": response})